In [1]:
import json

test_path = "datasets/Wikilingual-dataset/test.json"  # chỉnh lại nếu tên khác

with open(test_path, "r", encoding="utf-8") as f:
    data = json.load(f)

max_len = 0
max_id = None
lengths = []

for sid, sample in data.items():
    target = sample["target"]
    length = len(target.split())   # đếm token theo space
    lengths.append(length)

    if length > max_len:
        max_len = length
        max_id = sid

print("Số sample:", len(lengths))
print("Target dài nhất:", max_len, "tokens")
print("Sample id:", max_id)

# thống kê thêm
lengths.sort()
print("95% percentile:", lengths[int(0.95 * len(lengths))])
print("99% percentile:", lengths[int(0.99 * len(lengths))])


Số sample: 3789
Target dài nhất: 192 tokens
Sample id: wikilingua_multilingual-test-23022
95% percentile: 87
99% percentile: 123


In [2]:
import json
import numpy as np

vietnews_test_path = "datasets/Vietnews-dataset/test.json"  # chỉnh nếu khác

with open(vietnews_test_path, "r", encoding="utf-8") as f:
    data = json.load(f)

lengths = []
max_len = 0
max_id = None

for sid, sample in data.items():
    target = sample["target"]
    l = len(target.split())
    lengths.append(l)

    if l > max_len:
        max_len = l
        max_id = sid

lengths = np.array(lengths)

print("Số sample:", len(lengths))
print("Target dài nhất:", max_len, "tokens")
print("Sample id dài nhất:", max_id)
print("Target:", data[max_id]["target"])
print()
print("Mean:", lengths.mean())
print("Median:", np.median(lengths))
print("95% percentile:", np.percentile(lengths, 95))
print("99% percentile:", np.percentile(lengths, 99))


Số sample: 22644
Target dài nhất: 45 tokens
Sample id dài nhất: 13593
Target: Không thể bỏ lỡ : Người đàn ông dùng súng tự chế tự tử vì mâu thuẫn với gia đình , Kiểm lâm bỏng mặt vì chữa cháy rừng ở Thừa Thiên - Huế , Xe khách biển Lào đi lùi trên đường Hà Nội

Mean: 13.85978625684508
Median: 14.0
95% percentile: 20.0
99% percentile: 24.0


In [ ]:
# Dummy input tensor (batch_size=1, seq_len=512)
dummy_input = torch.randint(low=0, high=vocab.vocab_size, size=(1, 512))

# Wrap model
wrapped_model = SummarizationWrapper(model)

# Trace
scripted_model = torch.jit.trace(wrapped_model, dummy_input)

# Lưu TorchScript
torch.jit.save(scripted_model, "checkpoints/LongformerEncoderDecoder_Wikilingual/longformer_summarizer.pt")
print("TorchScript model saved!")
